In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor

In [7]:
Product_csv = pd.read_csv('Product_range.csv')
Transactions_csv = pd.read_csv('Transactions.csv')
print(Product_csv.columns)
print(Transactions_csv.columns)


Index(['Product_code', 'Vendor_code', 'Name', 'Retail_price', 'Base_unit',
       'Country_of_Origin', 'Size', 'ABV'],
      dtype='object')
Index(['Date_and_time_of_unloading', 'Product_code', 'Amount', 'Sale_amount',
       'Discount_amount', 'Profit', 'Percentage_markup',
       'Discount_percentage'],
      dtype='object')


In [25]:
df_total = pd.merge(Transactions_csv, Product_csv, on='Product_code')
df_total.to_csv('Transactions_with_Product_data.csv')